# Stochastic Optimization with Heavy-Tailed Noise via Accelerated Gradient Clipping

### 1. Постановка задача

$$\min_{x \in \mathbb{R}^n}f(x), \quad f(x) = \mathbb{E}_{\xi}[f(x, \xi)],$$
где $f(x)$
  - выпуклая
  - гладкая
  - $\forall x \in \mathbb{R}^n \rightarrow \exists \nabla f(x, \xi): \mathbb{E}_{\xi}[\nabla f(x, \xi)] = \nabla f(x), \quad \mathbb{E}_{\xi}[\|\nabla f(x, \xi) - \nabla f(x)\|^2_2] \leq \sigma^2$


**Опр.** Случайный вектор $\xi$ принадлежит **распределению с легкими хвостами**, если
  1. $\mathbb{E}[\xi]$
  2. $\forall b > 0 \rightarrow \mathbb{P}\{\|\xi - \mathbb{E}[\xi]\|_2 > b\} \leq 2 exp(-\frac{b^2}{2\sigma^2})$ 

Что эквивалентно условию [1]

$$\mathbb{E}[exp(\|\xi - \mathbb{E}[\xi]\|_2^2 / {\sigma^2})] \leq exp(1)$$

### Базовая терминология

**Опр.** Пусть дана непрерывно дифференцируемая на $\mathbb{R}^n$ функция $ f(x) : \mathbb{R}^n \longrightarrow \mathbb{R} $ . Будем говорить, что она является **выпуклой**, если для любых x, y $\in \mathbb{R}^n$ выполнено
$$f(y) \geq f(x) + \langle \nabla f(x), y - x \rangle$$

**Опр.** Пусть дана непрерывно дифференцируемая на $\mathbb{R}^n$ функция  $ f(x) : \mathbb{R}^n \longrightarrow \mathbb{R} $. Будем говорить, что данная функция имеет L-Липшицев градиент (говорить, что она является **L-гладкой**), если для любых x, y $\in \mathbb{R}^n$ выполнено

$$\|\nabla f(x) - \nabla f(y)\|_2 \leq L\|x - y\|_2$$

### Ссылки на источники

[0] Stochastic Optimization with Heavy-Tailed Noise via Accelerated Gradient Clipping (Eduard Gorbunov, Marina Danilova, Alexander Gasnikov)

[1] Лемма 2, Chi Jin, Praneeth Netrapalli, Rong Ge, Sham M Kakade, and Michael I Jordan. A short
note on concentration inequalities for random vectors with subgaussian norm. arXiv preprint
arXiv:1902.03736, 2019